In [3]:
import pandas as pd

try:
    df = pd.read_csv('/content/Train.csv', error_bad_lines=False, warn_bad_lines=True)
except Exception as e:
    print("An error occurred:", e)
# Read lines around the problematic area to inspect them
with open('/content/Train.csv', 'r') as file:
    for i, line in enumerate(file):
        if i in range(21470, 21480):  # Adjust the range as necessary
            print(f"Line {i}: {line}")
# Read lines around the problematic area to inspect them
with open('/content/Train.csv', 'r') as file:
    for i, line in enumerate(file):
        if i in range(21470, 21480):  # Adjust the range as necessary
            print(f"Line {i}: {line}")
import csv

# Use csv.reader to handle complex CSVs
with open('/content/Train.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    rows = [row for row in reader]

# Convert to DataFrame
df = pd.DataFrame(rows[1:], columns=rows[0])


An error occurred: read_csv() got an unexpected keyword argument 'error_bad_lines'
Line 21470: "I don't understand why making remakes has become the trend. Every remake I have ever seen is awful, and this is no exception. If any of you have seen the quote from Ben Jones, that it is a ""sleazy"" piece of trash, he is quite right. Why they would take a wonderful television show, which I loved, have never missed an episode, and own seasons 1-4 on DVD, and ruin it, I'll never know. The television show was a family show, and although Daisy has the body, it was really flaunted, or even addressed in the show, save the outfits. A family show has been turned in to a dirty piece of garbage, and I wouldn't recommend anyone go see it. Another thing I didn't like was that John Schneider and Tom Wopat are excellent actors (along with the rest of the original cast), and they are also extremely cute. The new Bo and Luke are not even a little cute. That was one of the drawers for the show. The casting 

In [5]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Ensuring CUDA is available and setting device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)

# Load the dataset
dataset = load_dataset('csv', data_files={'train': '/content/Train.csv'})
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,  # Adjust based on GPU memory
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    fp16=True,  # Enable mixed precision
    load_best_model_at_end=True,  # Load the best model at the end of training based on validation loss
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],  # Make sure you have a test split or adjust accordingly
)

trainer.train()


In [ ]:
from datasets import concatenate_datasets

# Assuming 'original_dataset' is your original pre-training dataset loaded similarly
original_dataset = load_dataset('path_to_original_dataset', split='train')
instruction_based_dataset = load_dataset('csv', data_files={'train': 'instruction_based_imdb_dataset.csv'})

# Combine datasets
combined_dataset = concatenate_datasets([original_dataset, instruction_based_dataset['train']])

# Tokenize the combined dataset
tokenized_combined_datasets = combined_dataset.map(tokenize_function, batched=True)

# Fine-tune on the combined dataset
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_combined_datasets,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./fine_tuned_distilbert_combined')


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the evaluation trainer
evaluation_trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics
)

# Evaluate the model
results = evaluation_trainer.evaluate(tokenized_datasets['test'])  # assuming a test split is available
print(results)
